In [1]:
import altair as alt
import numpy as np
import math
import polars as pl
import requests

from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.palettes import Inferno256
from bokeh.plotting import figure, show
from bokeh.tile_providers import OSM, get_provider
from bokeh.io import output_notebook

tile_provider = get_provider(OSM)
color_mapper = LinearColorMapper(palette=Inferno256)
output_notebook()

Loading BokehJS ...

In [2]:
positions = pl.read_csv("positions.csv")

In [3]:
# https://wiki.openstreetmap.org/wiki/Mercator#Python
MERCATOR_RADIUS = 6378137

positions = positions.filter(
    (pl.col("x") > -90) & (pl.col("x") < 90) & (pl.col("y") > -180) & (pl.col("y") < 180)
).sort(
    ["k", "timestamp"]
).with_columns([
    (pl.col("timestamp")+"Z").str.strptime(pl.Datetime, fmt="%+", strict=False),
    np.deg2rad(pl.col("x")).alias("x_rad"),
    np.deg2rad(pl.col("y")).alias("y_rad"),
]).with_columns([
    pl.internals.expr.ExprDateTimeNameSpace.seconds(pl.col("timestamp")-pl.col("timestamp").shift(1)).alias("time_diff_s"),
    ((pl.col("x_rad")/2 + math.pi/4).tan().log() * MERCATOR_RADIUS).alias("y_mercator"),
    (pl.col("y_rad") * MERCATOR_RADIUS).alias("x_mercator"),
])

In [4]:
# https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
positions['distance_km'] = positions.select([
    pl.col("x_rad").alias("x"),
    pl.col("y_rad").alias("y"),
    pl.col("x_rad").shift(1).alias("prev_x"),
    pl.col("y_rad").shift(1).alias("prev_y")
]).with_columns([
    (pl.col("x") - pl.col("prev_x")).alias("d_lat"),
    (pl.col("y") - pl.col("prev_y")).alias("d_lon"),
]).with_columns([
    ((pl.col("d_lat") / 2).sin().pow(2) + (pl.col("d_lon") / 2).sin().pow(2) * pl.col("prev_x").cos() * pl.col("x").cos()).alias("a")
]).with_columns([
    ((pl.col("a").sqrt() / (1-pl.col("a")).sqrt()).arctan() * 2 * 6373).alias("distance_km")
])['distance_km']

In [5]:
positions = positions.with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("time_diff_s")).alias("time_diff_s")
).with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("distance_km")).alias("distance_km")
).with_column(
    (pl.col("distance_km") / pl.col("time_diff_s") * 3600).alias("speed_km_h")
)

In [6]:
# positions.groupby('k').agg(pl.col("k").count()).sort(by="k_count", reverse=True)

In [7]:
# TODO verify if this cutoff works fine
# TODO do it for all "k"

WINDOW_OBS = 40
CUTOFF_KM_H = 5

test = positions.filter(pl.col("k") == 19706305)
test = test.with_columns([
    pl.col("distance_km").cumsum().alias("total_distance"),
    pl.col("speed_km_h").rolling_max(window_size=WINDOW_OBS).alias("rolling_max_speed"),
]).with_columns([
    (pl.col("rolling_max_speed") <= CUTOFF_KM_H).cumsum().shift(-(WINDOW_OBS-1)).alias("cutoff"),
]).filter(
    pl.col("cutoff") <= 1
)

In [8]:
# https://docs.bokeh.org/en/latest/docs/reference/models/formatters.html?highlight=datetimetickformatter#bokeh.models.DatetimeTickFormatter
p = figure(title="Simple line example", x_axis_type = "datetime", x_axis_label='x', y_axis_label='y')
p.line(
    test["timestamp"],
#     test["total_distance"],
    test["speed_km_h"],
    line_width=2
)
show(p)

In [9]:
source = ColumnDataSource(
    data=test.select([
        pl.col("x_mercator"),
        pl.col("y_mercator"),
        pl.col("timestamp"),
        pl.col("speed_km_h"),
        (pl.col("speed_km_h")+1).log().alias("speed_map_scale"),
    ]).to_dict(as_series=False)
)

p = figure(
    x_range=(1875733.4, 1914695.2), 
    y_range=(6621293.7, 6674532.7),
    x_axis_type="mercator",
    y_axis_type="mercator"
)
p.add_tile(tile_provider)
p.circle(
    source=source,
    x="x_mercator",
    y="y_mercator",
    size=10,
#     fill_color={"field": "speed_map_scale", "transform": color_mapper},
    fill_color={"field": "timestamp", "transform": color_mapper},
    fill_alpha=0.8
)
show(p)
# https://docs.bokeh.org/en/latest/docs/user_guide/geo.html?highlight=geojson

In [10]:
rides = positions.groupby('k').agg([
    pl.col("name").first().alias("name"),
    (pl.col("distance_km").sum() / pl.col("time_diff_s").sum() * 3600).alias("speed_km_h_avg")
])

line_stats = rides.groupby("name").agg([
    pl.col("k").n_unique().alias("rides"),
    pl.col("speed_km_h_avg").median(),
]).sort("speed_km_h_avg_median")

source = ColumnDataSource(
    data=line_stats.to_dict(as_series=False)
)

p = figure(
    title="Categorical Dot Plot",
    tools="",
    toolbar_location=None,
    y_range=line_stats["name"].to_list(),
    x_range=[0, 50]
)
p.segment(source=source, x0=0, y0="name", x1="speed_km_h_avg_median", y1="name", line_width=2, line_color="green", )
p.circle(source=source, x="speed_km_h_avg_median", y="name", size=15, fill_color="orange", line_color="green", line_width=3, )

show(p)